In [2]:
import pandas as pd
import os

from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

from sqlalchemy import text
import sqlite3


In [10]:
df = pd.read_json('data/lecture_data.json')
df.head()


df_renamed = df.rename(columns = {'prefix' : 'department_abbrv', 'GenEd Requirements' : 'credit_type', 'mean' : 'mean_gpa',
                                  'Course Code' : 'course_id', 'Credits' : 'credits', 'Term' : 'quarter_offered',
                                  'Rating' : 'teacher_rating', 'Start' : 'start_time', 'End' : 'end_time', 'Days' : 'meeting_days',
                                  'Instructor' : 'professor_name'})

df_final = df_renamed[['department_abbrv', 'course_id', 'credits', 'quarter_offered', 'mean_gpa', 'teacher_rating', 'start_time', 'end_time', 'meeting_days', 'professor_name', 'credit_type']]

In [12]:
df_final.groupby(['course_id', 'professor_name']).value_counts().sort_values(ascending=False)

course_id   professor_name            department_abbrv  credits  quarter_offered  mean_gpa  teacher_rating  start_time  end_time  meeting_days  credit_type
MICROM 302  Karen R Thickman          MICROM            2        Spring 2025      2.778167  2.4             13:30       15:20     TTh           NSc            8
                                                                                                            15:30       17:20     TTh           NSc            8
HUM    498  MariaElena Garcia         HUM               6        Summer 2025      3.929474  4.7             09:10       12:20     WThF          SSc, A&H       4
MICROM 402  Kendall M Gray            MICROM            3        Spring 2025      2.961317  4.0             14:30       16:20     MWF           NSc            4
CSE    351  Ruth Anderson             CSE               4        Spring 2025      3.531082  4.2             11:30       12:20     MWF                          4
                                       

In [3]:
df_final['course_id'] = df_final['course_id'].str.extract(r'(\d+)', expand=False).astype(int)

C:\Users\lucas\AppData\Local\Temp\ipykernel_660\3716092654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['course_id'] = df_final['course_id'].str.extract(r'(\d+)', expand=False).astype(int)


In [4]:
df_final.head()

,department_abbrv,course_id,credits,quarter_offered,mean_gpa,teacher_rating,start_time,end_time,meeting_days,professor_name,credit_type
0,AFRAM,150,5,Spring 2025,3.440223,NaN,16:30,18:20,TTh,Alekzandr Wray,SSc
1,AFRAM,272,5,Spring 2025,3.498758,NaN,16:30,18:20,MW,Alekzandr Wray,"SSc, DIV"
2,AFRAM,318,5,Spring 2025,3.486667,4.6,13:30,15:20,TTh,Sonnet H. Retman,"A&H, DIV"
3,AFRAM,330,5,Spring 2025,3.543902,4.6,16:00,17:20,TTh,Sonnet H. Retman,A&H
4,AFRAM,499,1-5,Summer 2025,3.906667,NaN,None,None,None,None,


In [27]:
filtered_df = df_final[
    (df_final['department_abbrv'] == 'CSE') & 
    (df_final['course_id'].str[4:].astype(int) >= 200) & 
    (df_final['teacher_rating'] > 3)
][['department_abbrv', 'course_id', 'teacher_rating']]

ValueError: invalid literal for int() with base 10: 'M  150'

In [24]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)  # Wider display
pd.set_option('display.max_colwidth', 500) 

In [17]:
df.head()

,Course Code,Course Description,Course Title,Credits,prefix,Start,End,Instructor,Rating,mean,Term,Building,GenEd Requirements,Days,Section,Registration Code
0,AFRAM 150,Introductory survey of topics and problems in ...,Introduction to African American History,5,AFRAM,16:30,18:20,Alekzandr Wray,NaN,3.440223,Spring 2025,CMU,SSc,TTh,A,10179
1,AFRAM 272,"Reconstruction and its aftermath, the Agrarian...",History of the South Since the Civil War,5,AFRAM,16:30,18:20,Alekzandr Wray,NaN,3.498758,Spring 2025,THO,"SSc, DIV",MW,A,10180
2,AFRAM 318,Considers how generic forms and conventions ha...,Black Literary Genres,5,AFRAM,13:30,15:20,Sonnet H. Retman,4.6,3.486667,Spring 2025,MGH,"A&H, DIV",TTh,A,10181
3,AFRAM 330,Focuses on cultural expressions created by peo...,"Music, Folklore, and Performance in Black Society",5,AFRAM,16:00,17:20,Sonnet H. Retman,4.6,3.543902,Spring 2025,CHL,A&H,TTh,A,10182
4,AFRAM 499,Identification and investigation of the proble...,Independent Study and Research ...,1-5,AFRAM,None,None,None,NaN,3.906667,Summer 2025,None,,None,A,10042


In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('databases/final_temp.db')
cursor = conn.cursor()

# Define table name
table_name = 'class_data'  # Adjust this to match your actual table name


# Insert DataFrame into the SQLite table
df_final.to_sql(table_name, conn, index=False)

NameError: name 'df_final' is not defined

In [4]:
# Replace with your database URL
engine = create_engine("sqlite:///databases/all_data.db")  # Example for SQLite

# Create MetaData object
metadata = MetaData()
metadata.reflect(bind=engine)

# Print all tables and their columns
for table_name, table in metadata.tables.items():
    print(f"Table: {table_name}")
    for column in table.columns:
        print(f"  - {column.name} ({column.type})")


Table: courses
  - Campus (TEXT)
  - Term (TEXT)
  - Course Code (TEXT)
  - Course Title (TEXT)
  - Course Description (TEXT)
  - Activity Type (TEXT)
  - Section (TEXT)
  - Credits (TEXT)
  - Days (TEXT)
  - Time (TEXT)
  - Building (TEXT)
  - Room (TEXT)
  - Instructor (TEXT)
  - Credit (TEXT)
  - GenEd Requirements (TEXT)
  - Registration Code (TEXT)
  - enrollCount (TEXT)
  - enrollMaximum (TEXT)
  - myPlanLink (TEXT)
  - index (TEXT)
  - mean (REAL)
  - median (REAL)
  - sd (REAL)
  - fourpointo_per (REAL)
  - 0 (REAL)
  - 1 (REAL)
  - 2 (REAL)
  - 3 (REAL)
  - 4 (REAL)
  - 5 (REAL)
  - 6 (REAL)
  - 7 (REAL)
  - 8 (REAL)
  - 9 (REAL)
  - 10 (REAL)
  - 11 (REAL)
  - 12 (REAL)
  - 13 (REAL)
  - 14 (REAL)
  - 15 (REAL)
  - 16 (REAL)
  - 17 (REAL)
  - 18 (REAL)
  - 19 (REAL)
  - 20 (REAL)
  - 21 (REAL)
  - 22 (REAL)
  - 23 (REAL)
  - 24 (REAL)
  - 25 (REAL)
  - 26 (REAL)
  - 27 (REAL)
  - 28 (REAL)
  - 29 (REAL)
  - 30 (REAL)
  - 31 (REAL)
  - 32 (REAL)
  - 33 (REAL)
  - 34 (REAL)
  -

In [15]:
sql = '''SELECT *\nFROM courses\nWHERE 'Course Code' = 'CSE   331' '''


with engine.connect() as con:
        rows = con.execute(text(sql))

for row in rows:
        print(row)     
